In [3]:
import os
import konlpy
import gensim
import pandas as pd

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


### 1) 데이터 준비 와 확인

In [11]:
file_path = os.getenv('HOME') + '/aiffel/aiffel_quest/mini_quest_240322/data/'

train_data = pd.read_table(file_path + 'ratings_train.txt')
test_data = pd.read_table(file_path + 'ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 2) 데이터로더 구성

In [ ]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']



### 3) 모델 구성을 위한 데이터 분석 및 가공

In [ ]:
# 데이터셋 내 만장 길이 분포

In [ ]:
# 적절한 최대 문장 길이 지정

In [ ]:
# keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

### 4) 모델 구성 및 validation set 구성

In [ ]:
# 모델은 3가지 이상 다양하게 구성하여 실험

### 5) 모델 훈련 개시

### 6) Loss, Accuracy 그래프 시각화

### 7) 학습된 Embedding 레이어 분석

### 8) 한국어 Word2Vec 임베딩 활용하여 성능 개선
- 한국어 Word2Vec은 /data 폴더 안에 있는 word2vec_ko.model을 활용하세요.
- 한국어 Word2Vec을 활용할 때는 load_word2vec_format() 형태가 아닌 load() 형태로 모델을 불러와주세요. 또한 - 모델을 활용할 때에는 아래 예시와 같이 .wv를 붙여서 활용합니다. 좀더 자세한 활용법에 대해선 다음 링크들을 참조해주세요. [참고 링크1](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#storing-and-loading-models), [참고 링크2](https://radimrehurek.com/gensim/models/keyedvectors.html)

In [ ]:
# 예시 코드
from gensim.models.keyedvectors import Word2VecKeyedVectors
word_vectors = Word2VecKeyedVectors.load(word2vec_file_path)
vector = word_vectors.wv[‘끝’]